# Training Metrics Visualization: SuperPoint+LightGlue (Spirals Legacy)

Run: `lightglue_spirals_v1_20251210_002245`

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator

sns.set_theme(style="whitegrid")
plt.rcParams["figure.dpi"] = 120

In [ ]:
# Path to the event file
log_dir = "/home/guillemc/dev/LuPNT-private/output/2025_FeatureMatching/training_old/lightglue_spirals_v1_20251210_002245"
event_file = [f for f in os.listdir(log_dir) if "events.out.tfevents" in f][0]
event_path = os.path.join(log_dir, event_file)
print(f"Reading events from: {event_path}")

In [ ]:
# Load the event accumulator
ea = EventAccumulator(event_path)
ea.Reload()

# Get list of available tags
tags = ea.Tags()["scalars"]
print("Available Tags:", tags[:10], "...")

In [ ]:
# Extract scalars into a DataFrame
data = []
for tag in tags:
    events = ea.Scalars(tag)
    for event in events:
        data.append(
            {
                "Step": event.step,
                "Value": event.value,
                "Metric": tag,
                "WallTime": event.wall_time,
            }
        )

df = pd.DataFrame(data)
df.head()

## 1. Metric Groups

In [ ]:
train_tags = [
    t for t in tags if "training" in t.lower() and "lr" not in t and "epoch" not in t
]
val_tags = [
    t
    for t in tags
    if "checkpoint" in t.lower()
    or "val" in t.lower()
    or "recall" in t.lower()
    or "prec" in t.lower()
]
val_tags = [t for t in val_tags if "train" not in t]

# --- 1. Total Loss Comparison (Train vs Val) ---
train_total = df[df["Metric"] == "training//total"].copy()
val_total = df[df["Metric"] == "val/loss/total"].copy()

if not train_total.empty and not val_total.empty:
    train_total["Split"] = "Training"
    val_total["Split"] = "Validation"
    # Validation often has fewer steps; markers help visualize the points
    combined = pd.concat([train_total, val_total])

    plt.figure(figsize=(12, 6))
    sns.lineplot(
        data=combined, x="Step", y="Value", hue="Split", style="Split", markers=True
    )
    plt.title("Total Loss: Training vs Validation")
    plt.yscale("log")
    plt.ylabel("Loss")
    plt.grid(True, which="minor", linewidth=0.5, alpha=0.3)
    plt.show()


# --- 2. Detailed Training Metrics ---
comp_tags = [t for t in train_tags if "total" not in t and "num" not in t]
if comp_tags:
    plt.figure(figsize=(12, 5))
    sns.lineplot(
        data=df[df["Metric"].isin(comp_tags)], x="Step", y="Value", hue="Metric"
    )
    plt.title("Training: Loss Components")
    plt.yscale("log")
    plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
    plt.tight_layout()
    plt.show()

count_tags = [t for t in train_tags if "num" in t]
if count_tags:
    plt.figure(figsize=(12, 5))
    sns.lineplot(
        data=df[df["Metric"].isin(count_tags)], x="Step", y="Value", hue="Metric"
    )
    plt.title("Training: Matchable/Unmatchable Keypoints")
    plt.show()

# --- 3. Detailed Validation Metrics ---
val_perf_tags = [t for t in val_tags if "loss" not in t]
if val_perf_tags:
    plt.figure(figsize=(12, 5))
    sns.lineplot(
        data=df[df["Metric"].isin(val_perf_tags)],
        x="Step",
        y="Value",
        hue="Metric",
        marker="o",
    )
    plt.title("Validation: Performance Metrics (Recall, Accuracy, mAP)")
    plt.legend(loc="lower right")
    plt.tight_layout()
    plt.show()

val_loss_comps = [
    t for t in val_tags if "loss" in t and "total" not in t and "num" not in t
]
if val_loss_comps:
    plt.figure(figsize=(12, 5))
    sns.lineplot(
        data=df[df["Metric"].isin(val_loss_comps)],
        x="Step",
        y="Value",
        hue="Metric",
        marker="o",
    )
    plt.title("Validation: Loss Components")
    plt.yscale("log")
    plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
    plt.tight_layout()
    plt.show()